In [1]:
import os

In [ ]:
%pwd  #going into the research folder

'c:\\Users\\sheet\\Desktop\\MachineLearning\\Briefly\\research'

In [3]:
os.chdir("../")   #we have to go out 1 folder before

In [4]:
%pwd

'c:\\Users\\sheet\\Desktop\\MachineLearning\\Briefly'

In [ ]:
from dataclasses import dataclass     #this goes in the entity file
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    


In [ ]:
import sys
import os

sys.path.append(os.path.abspath("src"))

from Briefly.constants import *  # Now it should work
from Briefly.utils.common import  read_yaml, create_directories
# this goes in configuration

In [ ]:
class ConfigurationManager:           #This goes into configuration.py
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        """Initialize Configuration Manager and load YAML configs."""
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Ensure artifacts directory exists
        create_directories([self.config["artifacts_root"]])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """Retrieve Data Ingestion Configuration."""
        config = self.config["data_ingestion"]  # Use dictionary syntax!

        create_directories([config["root_dir"]])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config["root_dir"],
            source_URL=config["source_URL"],
            local_data_file=config["local_data_file"],
            unzip_dir=config["unzip_dir"],
        )
        return data_ingestion_config


In [25]:
import os
import urllib.request as request
import zipfile
from Briefly.logging import logger
from Briefly.utils.common import get_size


In [28]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):  # ✅ Correct attribute name
            filename, headers = request.urlretrieve(
            url=self.config.source_URL,
            filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """"
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns  None"
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok= True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [29]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-03-30 12:58:41,552: INFO: yaml file: config\config.yaml loaded successfully]
[2025-03-30 12:58:41,556: INFO: yaml file: params.yaml loaded successfully]
[2025-03-30 12:58:41,558: INFO: created directory at: artifacts]
[2025-03-30 12:58:41,560: INFO: created directory at: artifacts/data_ingestion]
[2025-03-30 12:58:58,273: INFO: artifacts/data_ingestion/data.zip downloaded! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: AC04:2EB443:421C1F:AFF09B:67E8F2AB
Accept-Ranges: bytes
Date: Sun, 30 Mar 2025 07:28:44 GMT
Via: 1.1 varnish
X-Served-By: cache-del21748-DEL
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1743319723.3